# PreProcessing CSV Files for use with Tensorflow

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

In [2]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = "Data/merge_data"

In [3]:
df = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load(["Data/SS_POOLS.csv", "Data/SD_CHUNK_LOCATIONS.csv", "Data/ARCHIVE_OBJECTS.csv"])

# df.printSchema()
# df.columns

The cell below takes the four columns that we want and writes casts them as integers. We then select each of the four features and write the new data frame to the desired folder.

In [5]:
df.withColumn("POOLID", df["POOLID"].cast("int"))
df.withColumn("SIZE", df["SIZE"].cast("int"))
df.withColumn("OFFSET", df["OFFSET"].cast("int"))
df.withColumn("LENGTH", df["LENGTH"].cast("int"))

df.select("POOLID", "SIZE", "OFFSET", "LENGTH").write.options(header='true').format('com.databricks.spark.csv').save("Data/merge_data/4_features")

The cell below then goes and reads the CSV files generated by the cell above and prints out all the new column names and types to make sure we outputted what we wanted.

In [6]:
test_df = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load("Data/merge_data/4_features/*.csv")
test_df.columns

['POOLID', 'SIZE', 'OFFSET', 'LENGTH']

In [10]:
sc.stop()